# APIM ❤️ OpenAI

## Private connectivity lab
![flow](./architecture.png)

Playground to show how to create a private network for consuming LLMs from `AI Services`.
This lab demonstrates how to create a private network for consuming LLMs from `AI Services` using `Private Link Services`, `Azure API Management (APIM)`, and `Azure Front Door`.

Notes:
- `Azure OpenAI` is only accessible through `Private Endpoints`. Public network access is disabled.
- `Azure API Management` is integrated in the private network and is used to manage the traffic to the `Azure OpenAI` service through `Private Endpoints`.
- `Azure API Management` is not accessible from a public network. The only access is through `Azure Front Door`.
- `Azure Front Door` manages the traffic to the `Azure API Management` service through `Private Link Service`.
- **`Azure Front Door` is the only publicly-accessible service.**

### Prerequisites
- [Python 3.12 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [5]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-25-{deployment_name}" # change the name to match your naming style
resource_group_location = "francecentral"

apim_sku = 'Standardv2'

# Prioritize East US until exhaustion (simulate PTU with TPM), then equally distribute between Sweden and West US (consumption fallback)
openai_resources = [
    {"name": "openai1", "capacity": 20, "location": "eastus", "priority": 1},
    {"name": "openai2", "capacity": 20, "location": "swedencentral", "priority": 2, "weight": 50},
    {"name": "openai3", "capacity": 20, "location": "westus", "priority": 2, "weight": 50}
]

openai_deployment_name = "gpt-4o-mini"
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_capacity = 8
openai_model_sku = 'Standard'
openai_api_version = "2024-02-01"

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 22:36:00.358089 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [2]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 10:00:34.571530 :2s]
👉🏽 Current user: admin@MngEnvMCAP784683.onmicrosoft.com
👉🏽 Tenant ID: 93139d1e-a3c1-4d78-9ed5-878be090eba4
👉🏽 Subscription ID: dcef7009-6b94-4382-afdc-17eb160d709a


In [13]:
vmSshPublicKey = "" # read file at path C:/Users/hodellai/.ssh/id_rsa.pub
if os.path.exists("C:/Users/hodellai/.ssh/id_rsa.pub"):
    with open("C:/Users/hodellai/.ssh/id_rsa.pub", "r") as file:
        vmSshPublicKey = file.read().strip()

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations.

In [18]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "apimPublicNetworkAccess": { "value": "Enabled" },
        "vmSshPublicKey": { "value": vmSshPublicKey },
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-25-private-connectivity 
👉🏽 Using existing resource group 'lab-25-private-connectivity'
⚙️ Running: az deployment group create --name private-connectivity --resource-group lab-25-private-connectivity --template-file main.bicep --parameters params.json 
✅ Deployment 'private-connectivity' succeeded ⌚ 22:08:03.698415 :47s]


<a id='3'></a>
### 3️⃣ Approve Front Door private link connection to APIM

In the deployed Bicep template, Azure Front Door will establish a private link connection to the API Management service. This connection should be approved. Run the following command to approve the connection.

In [4]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_resource_id = utils.get_deployment_output(output, 'apimResourceId', 'apimResourceId')

    outputPls = utils.run(f"az network private-endpoint-connection list --id {apim_resource_id} --query [?properties.privateLinkServiceConnectionState.status=='Pending'].id --output tsv")
    
    if outputPls.success:
        pls_connection_id = outputPls.text
        print(pls_connection_id)
        utils.run(f"az network private-endpoint-connection approve --id {pls_connection_id} --description 'Approved'", f"Private Link Connection approved.", f"Failed to approve Private Link Connection: {pls_connection_id}")

⚙️ Running: az deployment group show --name private-connectivity -g lab-25-private-connectivity 
✅ Retrieved deployment: private-connectivity ⌚ 10:11:25.502154 :2s]
👉🏽 apimResourceId: /subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/lab-25-private-connectivity/providers/Microsoft.ApiManagement/service/apim-nrzp5vaasb57o
⚙️ Running: az network private-endpoint-connection list --id /subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/lab-25-private-connectivity/providers/Microsoft.ApiManagement/service/apim-nrzp5vaasb57o --query [?properties.privateLinkServiceConnectionState.status=='Pending'].id --output tsv 
/subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/lab-25-private-connectivity/providers/Microsoft.ApiManagement/service/apim-nrzp5vaasb57o/privateEndpointConnections/84021e47-5ae2-4c9e-bda2-775cd039f277

⚙️ Running: az network private-endpoint-connection approve --id /subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/

<a id='4'></a>
### 4️⃣ Disabling APIM public network access

As of May 2025, during the creation, the `APIM` service cannot disable the public network access. This behavior might change in the future. As a workaround, you can disable the public network access after the deployment is completed. To do that, you can run the Bicep template again after changing the `apimPublicNetworkAccess` parameter to `Disabled`. This will update the APIM service and disable the public network access.

In [5]:
bicep_parameters['parameters']['apimPublicNetworkAccess']['value'] = "Disabled"

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az deployment group create --name private-connectivity --resource-group lab-25-private-connectivity --template-file main.bicep --parameters params.json 
✅ Deployment 'private-connectivity' succeeded ⌚ 10:18:29.472448 :2s]


<a id='5'></a>
### 5️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [34]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    frontdoor_endpoint = utils.get_deployment_output(output, 'frontDoorEndpointHostName', 'Front Door Endpoint')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)

⚙️ Running: az deployment group show --name private-connectivity -g lab-25-private-connectivity 
✅ Retrieved deployment: private-connectivity ⌚ 16:39:15.139024 :2s]
👉🏽 Front Door Endpoint: afd-tcnui4s53u4bm-e3e6fpe2a2hga6gh.b01.azurefd.net
👉🏽 APIM API Gateway URL: https://apim-nrzp5vaasb57o.azure-api.net
👉🏽 APIM Subscription Key (masked): ****aaa5
✅ Retrieved deployment: private-connectivity ⌚ 16:39:15.139024 :2s]
👉🏽 Front Door Endpoint: afd-tcnui4s53u4bm-e3e6fpe2a2hga6gh.b01.azurefd.net
👉🏽 APIM API Gateway URL: https://apim-nrzp5vaasb57o.azure-api.net
👉🏽 APIM Subscription Key (masked): ****aaa5


<a id='6'></a>
### 6️⃣ 🧪 Test the API using a direct HTTP call through Frontdoor

Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

In [35]:
import requests

def call_api(resource_url):
    url = f"{resource_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}"

    messages = {"messages": [
        {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}

    response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)

    print(f"Response status code: {response.status_code}")

    if (response.status_code == 200):
        data = json.loads(response.text)
        print(f"💬 {data.get('choices')[0].get('message').get('content')}\n")
    else:
        print(f"{response.text}\n")

call_api(f"https://{frontdoor_endpoint}")

Response status code: 200
💬 Oh, sure! Just let me pull out my crystal ball… Oh wait, I can't see the time from here. How about you just check your phone or something? You know, that device you probably have glued to your hand?



<a id='7'></a>
### 7️⃣ 🧪 Test APIM API access through public network

APIM is not accessible from public network. The only way to access it is through Azure Front Door. This test should fail with a 403 error.

In [8]:
call_api(apim_resource_gateway_url)

Response status code: 403
{ "statusCode": 403, "message": "Request originated from client public IP address 74.241.135.213, public network access on this `Microsoft.ApiManagement/service/apim-nrzp5vaasb57o` is disabled. To connect to `Microsoft.ApiManagement/service/apim-nrzp5vaasb57o`, please use the Private Endpoint from inside your virtual network. To learn more https://aka.ms/apim-privateendpoint " }



<a id='8'></a>
### 8 🧪 Test APIM API access through private virtual network

APIM is accessible in the private virtual network through a `Private Endpoint`. This means that any resource within this network can communicate securely with the APIM service. To validate this behavior, the Bicep template creates an `Azure virtual machine` that will act as a `jumpbox` and a `Bastion Host` that will provide secure communication with the VM through `ssh`. 

Call the APIM API from within the VM. Navigate to the VM and connect to it using `Bastion` using the default username `azureuser` and password `@Aa123456789`.

The following code will generate a `cUrl` command that will test the API access. Copy its output and run it in the VM. 

This test should succeed with a 200 OK response.

In [ ]:
print(f'''curl -X POST "{apim_resource_gateway_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}" \\
  -H "Content-Type: application/json" \\
  -H "api-key: {apim_subscription_key}" \\
  -d '{{"messages": [ 
        {{"role": "system", "content": "You are a helpful assistant."}}, 
        {{"role": "user", "content": "What are 3 things to visit in Seattle?"}} 
      ]}}'
''')

curl -X POST "https://apim-nrzp5vaasb57o.azure-api.net/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-01" \
  -H "Content-Type: application/json" \
  -H "api-key: c866b266032040a499e77433094caaa5" \
  -d '{"messages": [ 
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": "What are 3 things to visit in Seattle?"} 
      ]}'



In [ ]:
! ssh -i C:/Users/hodellai/.ssh/id_rsa azureuser@4.233.66.65

In [91]:
! ssh -q -i C:/Users/hodellai/.ssh/id_rsa azureuser@4.233.66.65 "curl -X POST 'https://apim-nrzp5vaasb57o.azure-api.net/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-01' -H 'Content-Type: application/json' -H 'api-key: c866b266032040a499e77433094caaa5' -d '{\"messages\": [  {\"role\": \"system\", \"content\": \"You are a helpful assistant.\"},   {\"role\": \"user\", \"content\": \"What are 3 things to visit in Seattle?\"}      ]}' | jq .choices[0].message.content"

"Seattle is a vibrant city with plenty to explore! Here are three must-visit attractions:\n\n1. **Space Needle**: This iconic landmark offers stunning views of the Seattle skyline, Puget Sound, and the surrounding mountains. Visitors can take an elevator to the observation deck, which is especially beautiful at sunset or on clear days.\n\n2. **Pike Place Market**: One of the oldest continuously operating farmers' markets in the United States, Pike Place Market is a bustling hub of activity. You can find fresh produce, artisanal foods, handcrafted goods, and even the famous fish-throwing vendors. Don't forget to visit the original Starbucks store while you're there!\n\n3. **Chihuly Garden and Glass**: This exhibition showcases the mesmerizing glass artwork of Dale Chihuly. Located near the Space Needle, the gardens and indoor installations feature stunning colors and intricate designs, making it a unique and picturesque experience.\n\nThese attractions capture the essence of Seattle's c

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   158    0     0  100   158      0    152  0:00:01  0:00:01 --:--:--   152
100  2214  100  2056  100   158   1061     81  0:00:01  0:00:01 --:--:--  1143
100  2214  100  2056  100   158   1061     81  0:00:01  0:00:01 --:--:--  1143


In [90]:
# Use a Python-based approach to execute SSH and only capture the relevant output
import subprocess
import json

try:
    # Run SSH command but capture output programmatically
    ssh_command = [
        "ssh", "-q", "-i", "C:/Users/hodellai/.ssh/id_rsa", 
        "azureuser@4.233.66.65",
        """curl -s -X POST 'https://apim-nrzp5vaasb57o.azure-api.net/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-01' \
        -H 'Content-Type: application/json' \
        -H 'api-key: c866b266032040a499e77433094caaa5' \
        -d '{"messages": [{"role": "system", "content": "You are a helpful assistant."}, \
        {"role": "user", "content": "What are 3 things to visit in Seattle?"}]}' | jq -r .choices[0].message.content"""
    ]
    
    # Execute the command and capture output
    result = subprocess.run(ssh_command, capture_output=True, text=True)
    
    # Display only the OpenAI response (clean output)
    print("\n🤖 OpenAI Assistant's Response via SSH:\n")
    print(result.stdout.strip())
    
except Exception as e:
    print(f"Error executing SSH command: {e}")


🤖 OpenAI Assistant's Response via SSH:

Seattle is a vibrant city with a variety of attractions. Here are three must-visit places:

1. **Pike Place Market**: This iconic market is one of Seattle's most renowned attractions. You can explore a variety of stalls selling fresh produce, flowers, seafood, and unique crafts. Don't miss the famous fish-throwing tradition at the Pike Place Fish Market and be sure to check out the original Starbucks store.

2. **Space Needle**: A visit to Seattle isn't complete without taking in the views from the Space Needle. This iconic observation tower offers stunning panoramic views of the Seattle skyline, Puget Sound, and the surrounding mountains. The newly renovated observation deck features a glass floor, providing an exhilarating experience.

3. **Chihuly Garden and Glass**: Located right next to the Space Needle, this exhibition showcases the stunning glass artwork of Dale Chihuly. The vibrant colors and intricate designs of the glass sculptures bot

In [1]:
! az vm run-command invoke -g lab-25-private-connectivity -n vm-jumpbox --command-id RunShellScript --scripts "curl ifconf.me"

{
  "value": [
    {
      "code": "ProvisioningState/succeeded",
      "displayStatus": "Provisioning succeeded",
      "level": "Info",
      "message": "Enable succeeded: \n[stdout]\n4.233.66.65\n\n[stderr]\n  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total   Spent    Left  Speed\n\r  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r100    12  100    12    0     0     91      0 --:--:-- --:--:-- --:--:--    91\n",
      "time": null
    }
  ]
}


In [72]:
! az vm run-command invoke -g lab-25-private-connectivity -n vm-jumpbox --command-id RunShellScript --scripts "curl -X POST 'https://apim-nrzp5vaasb57o.azure-api.net/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-01' -H 'Content-Type: application/json' -H 'api-key: c866b266032040a499e77433094caaa5' -d '{\"messages\": [  {\"role\": \"system\", \"content\": \"You are a helpful assistant.\"},   {\"role\": \"user\", \"content\": \"What are 3 things to visit in Seattle?\"}      ]}'"

{
  "value": [
    {
      "code": "ProvisioningState/succeeded",
      "displayStatus": "Provisioning succeeded",
      "level": "Info",
      "message": "Enable succeeded: \n[stdout]\n{\"choices\":[{\"content_filter_results\":{\"hate\":{\"filtered\":false,\"severity\":\"safe\"},\"self_harm\":{\"filtered\":false,\"severity\":\"safe\"},\"sexual\":{\"filtered\":false,\"severity\":\"safe\"},\"violence\":{\"filtered\":false,\"severity\":\"safe\"}},\"finish_reason\":\"stop\",\"index\":0,\"message\":{\"annotations\":[],\"content\":\"Seattle is a vibrant city with a rich cultural scene and stunning natural beauty. Here are three must-visit attractions:\\n\\n1. **Pike Place Market**: This iconic market is famous for its fresh seafood, local produce, unique shops, and street performances. Be sure to check out the famous fish-throwing vendors, watch the artisans at work, and grab a coffee at the original Starbucks.\\n\\n2. **Space Needle**: A symbol of Seattle, the Space Needle offers breathtak

In [94]:
! az vm run-command invoke -g lab-25-private-connectivity -n vm-jumpbox --command-id RunShellScript --scripts "curl -X POST 'https://apim-nrzp5vaasb57o.azure-api.net/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-01' -H 'Content-Type: application/json' -H 'api-key: c866b266032040a499e77433094caaa5' -d '{\"messages\": [  {\"role\": \"system\", \"content\": \"You are a helpful assistant.\"},   {\"role\": \"user\", \"content\": \"What are 3 things to visit in Seattle?\"}      ]}' | jq .choices[0].message.content"

{
  "value": [
    {
      "code": "ProvisioningState/succeeded",
      "displayStatus": "Provisioning succeeded",
      "level": "Info",
      "message": "Enable succeeded: \n[stdout]\n\"Seattle is a vibrant city with many attractions. Here are three must-visit places:\\n\\n1. **Pike Place Market**: This iconic market is one of Seattle's most famous attractions. It features a variety of local vendors selling fresh produce, seafood, crafts, and specialty foods. Be sure to check out the famous fish-throwing vendors and visit the original Starbucks store.\\n\\n2. **Space Needle**: A symbol of Seattle, the Space Needle offers stunning views of the city and the surrounding mountains. You can take an elevator to the observation deck to enjoy panoramic views, and on clear days, you can see as far as Mount Rainier.\\n\\n3. **Chihuly Garden and Glass**: Located next to the Space Needle, this exhibition showcases the extraordinary glass artwork of artist Dale Chihuly. The indoor galleries and o

In [106]:
# Use utils.run instead of ! (shell command) to execute Azure CLI command
cmd = '''curl -X POST 'https://apim-nrzp5vaasb57o.azure-api.net/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-01' -H 'Content-Type: application/json' -H 'api-key: c866b266032040a499e77433094caaa5' -d '{\"messages\": [  {\"role\": \"system\", \"content\": \"You are a helpful assistant.\"},   {\"role\": \"user\", \"content\": \"What are 3 things to visit in Seattle?\"}      ]}' | jq .choices[0].message.content
'''

output = utils.run(f"az vm run-command invoke -g lab-25-private-connectivity -n vm-jumpbox --command-id RunShellScript --scripts {cmd}")
# Simplest possible approach to avoid any Unicode issues
if output.success and output.json_data:
    print(f"\n🤖 {output.json_data}")

⚙️ Running: az vm run-command invoke -g lab-25-private-connectivity -n vm-jumpbox --command-id RunShellScript --scripts curl -X POST 'https://apim-nrzp5vaasb57o.azure-api.net/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-01' -H 'Content-Type: application/json' -H 'api-key: c866b266032040a499e77433094caaa5' -d '{"messages": [  {"role": "system", "content": "You are a helpful assistant."},   {"role": "user", "content": "What are 3 things to visit in Seattle?"}      ]}' | jq .choices[0].message.content
 


In [ ]:
cmd = (
    "curl -X POST https://apim-nrzp5vaasb57o.azure-api.net/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-01 "
    "-H 'Content-Type: application/json' "
    "-H 'api-key: c866b266032040a499e77433094caaa5' "
    "-d '{\"messages\": [{\"role\": \"system\", \"content\": \"You are a helpful assistant.\"},{\"role\": \"user\", \"content\": \"What are 3 things to visit in Seattle?\"}]}'}"
)

az_command = f'az vm run-command invoke -g lab-25-private-connectivity -n vm-jumpbox --command-id RunShellScript --scripts "{cmd}"'

output = utils.run(az_command)

if output.success and output.json_data:
    print(f"\n🤖 {output.json_data}")
else:
    print(f"\n⚠️ Command failed. Output:\n{output}")


⚙️ Running: az vm run-command invoke -g lab-25-private-connectivity -n vm-jumpbox --command-id RunShellScript --scripts "curl -X POST https://apim-nrzp5vaasb57o.azure-api.net/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-01 -H 'Content-Type: application/json' -H 'api-key: c866b266032040a499e77433094caaa5' -d '{"messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "What are 3 things to visit in Seattle?"}]}'}" 

🤖 {'value': [{'code': 'ProvisioningState/succeeded', 'displayStatus': 'Provisioning succeeded', 'level': 'Info', 'message': 'Enable succeeded: \n[stdout]\n{\n  "error": {\n    "message": "[{\'type\': \'json_invalid\', \'loc\': (\'body\', 1), \'msg\': \'JSON decode error\', \'input\': {}, \'ctx\': {\'error\': \'Expecting property name enclosed in double quotes\'}}]",\n    "type": "invalid_request_error",\n    "param": null,\n    "code": null\n  }\n}\n[stderr]\n  % Total    % Received % Xferd  Average Speed   T

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.